In [1]:
import os
import numpy as np
import pandas as pd
from neupy import algorithms
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
import seaborn as sns

E:\Software\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Software\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Software\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Software\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1)

# UJI SKENARIO PERTAMA - SMOOTHING PARAMETER

Pada pengujian skenario pertama, akan dilakukan pengujian model PNN dengan variabel uji smoothing parameter. Dimana pada pengujian smoothing parameter ini dataset akan dibagi secara umum menjadi data train: data test dengan perbandingan 9:1. Kemudian untuk sudut dan jarak GLCM yang digunakan adalah sudut 0 dan jarak 1.

In [2]:
file = "GLCM extract/Combined/Non-Zoning/glcm_0_0.csv"

In [3]:
datasets = pd.read_csv(file)

In [4]:
datasets.drop(columns='img_paths')

,dissimilarity,homogeneity,contrast,ASM,energy,correlation,entropy,labels
0,4.940476,0.928288,947.937996,0.757715,0.870468,0.883208,3.821928,a
1,5.238839,0.935748,988.420387,0.787175,0.887229,0.851567,3.884184,a
2,9.313988,0.907800,1939.576885,0.717877,0.847276,0.778234,3.921928,a
3,7.342014,0.935135,1557.656994,0.821169,0.906184,0.660929,3.484184,a
4,9.650794,0.902062,1976.743056,0.724481,0.851164,0.719781,3.746439,a
...,...,...,...,...,...,...,...,...
2634,12.149058,0.878484,2493.052331,0.677331,0.823001,0.651378,3.184184,ya
2635,11.495784,0.794725,1839.395089,0.416249,0.645173,0.889178,4.121928,ya
2636,10.120784,0.789567,1570.168899,0.389454,0.624062,0.909622,4.121928,ya
2637,10.321925,0.793268,1520.020337,0.399891,0.632370,0.913381,3.821928,ya


In [5]:
datasets.describe()

,dissimilarity,homogeneity,contrast,ASM,energy,correlation,entropy
count,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000
mean,7.583715,0.917909,1537.414424,0.756904,0.866868,0.745515,3.393636
std,3.837481,0.043691,817.488180,0.117822,0.073789,0.127945,0.493272
min,1.089038,0.715397,163.271577,0.209960,0.458214,0.306753,1.191760
25%,4.311880,0.900582,843.384921,0.712011,0.843807,0.667253,3.130641
50%,7.394841,0.928539,1516.414683,0.789402,0.888483,0.758374,3.484184
75%,10.390997,0.947685,2108.890501,0.836753,0.914742,0.845068,3.784184
max,21.711310,0.983166,4729.200397,0.939957,0.969514,0.983693,4.121928


# VISUALIZATION 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
exclude_columns = ['contrast','ASM', 'energy', 'correlation', 'labels', 'img_paths']
select_columns = [x for x in datasets.columns if x not in exclude_columns]

In [ ]:
a4_dims = (11.7, 8.27)
for selected_feature in select_columns:
    plt.subplots(figsize=a4_dims)
    sns.boxplot(x=selected_feature, y="labels", data=datasets)
plt.show()

# END OF VISUALIZATION

In [6]:
X = datasets.loc[:, ['dissimilarity', 'homogeneity', 'entropy']].values
y = datasets.loc[:, 'labels'].values

In [7]:
X = preprocessing.normalize(X)

In [8]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)

In [9]:
kf

KFold(n_splits=10, random_state=1, shuffle=True)

In [12]:
char = ['あ', 'い','う', 'え','お']

In [14]:
char2 = ['あ', 'あ','う', 'え','お']

In [15]:
accuracy = metrics.accuracy_score(char, char2)
precision = metrics.precision_score(char, char2, average='micro')
recall = metrics.recall_score(char, char2, average='micro')

In [10]:
smoothing_parameter_results  = []
smoothing_recall = []
smoothing_precision = []
std_lists = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for std_val in std_lists:
    acc = 0
    prec = 0
    rec = 0
    counter = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pnn = algorithms.PNN(std=std_val, verbose=True)
        pnn.train(X_train, y_train)

        y_predicted = pnn.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_predicted)
        precision = metrics.precision_score(y_test, y_predicted, average='micro')
        recall = metrics.recall_score(y_test, y_predicted, average='micro')
        counter+=1
        acc+=accuracy
        prec+=precision
        rec+=recall
        print("Fold: ", counter, ", Accuracy: ",accuracy, " Recall: ",recall, " Precision: ",precision)
        print("-----------------------------------------------------------------------------------------------------------")
    acc=acc/counter
    prec=prec/counter
    rec=rec/counter
    print("\n\n\n\n\n\n")
    print("Smoothing Parameter value: ", std_val)
    print("Average accuracy: ", acc)
    smoothing_parameter_results.append(acc)
    smoothing_recall.append(rec)
    smoothing_precision.append(prec)
    print("=========================================================================================================")

Time: 0:00:00 |N/A%|                                           | ETA:  --:--:--


Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True



Fold:  1 , Accuracy:  0.19318181818181818  Recall:  0.19318181818181818  Precision:  0.19318181818181818
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True

Fold:  2 , Accuracy:  0.20075757575757575  Recall:  0.20075757575757575  Precision:  0.20075757575757575
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True

Fold:  3 , Accuracy:  0.23863636363636365  Recall:  0.23863636363636365  Precision:  0.23863636363636365
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True

Fold:  4 , Accuracy:  0.23106060606060605  


Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True

Fold:  7 , Accuracy:  0.22348484848484848  Recall:  0.22348484848484848  Precision:  0.22348484848484848
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True

Fold:  8 , Accuracy:  0.24621212121212122  Recall:  0.24621212121212122  Precision:  0.24621212121212122
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.1
[OPTION] verbose = True

Fold:  9 , Accuracy:  0.2159090909090909  Recall:  0.2159090909090909  Precision:  0.2159090909090909
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] b

Time: 0:00:00 |N/A%|                                           | ETA:  --:--:--

...

 4 , Accuracy:  0.17803030303030304  Recall:  0.17803030303030304  Precision:  0.17803030303030304
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.8
[OPTION] verbose = True

Fold:  5 , Accuracy:  0.17045454545454544  Recall:  0.17045454545454544  Precision:  0.17045454545454544
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.8
[OPTION] verbose = True

Fold:  6 , Accuracy:  0.19696969696969696  Recall:  0.19696969696969696  Precision:  0.19696969696969696
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.8
[OPTION] verbose = True

Fold:  7 , Accuracy:  0.20454545454545456  Recall

Time: 0:00:00 |N/A%|                                           | ETA:  --:--:--

Fold:  10 , Accuracy:  0.23193916349809887  Recall:  0.23193916349809887  Precision:  0.23193916349809887
-----------------------------------------------------------------------------------------------------------







Smoothing Parameter value:  0.8
Average accuracy:  0.2053908860467796

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.9
[OPTION] verbose = True

Fold:  1 , Accuracy:  0.1893939393939394  Recall:  0.1893939393939394  Precision:  0.1893939393939394
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.9
[OPTION] verbose = True

Fold:  2 , Accuracy:  0.20075757575757575  Recall:  0.20075757575757575  Precision:  0.20075757575757575
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std

Time: 0:00:00 |N/A%|                                           | ETA:  --:--:--

Fold:  5 , Accuracy:  0.17045454545454544  Recall:  0.17045454545454544  Precision:  0.17045454545454544
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.9
[OPTION] verbose = True

Fold:  6 , Accuracy:  0.19696969696969696  Recall:  0.19696969696969696  Precision:  0.19696969696969696
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.9
[OPTION] verbose = True

Fold:  7 , Accuracy:  0.20454545454545456  Recall:  0.20454545454545456  Precision:  0.20454545454545456
-----------------------------------------------------------------------------------------------------------

Main information

[ALGORITHM] PNN

[OPTION] batch_size = 128
[OPTION] std = 0.9
[OPTION] verbose = True

Fold:  8 , Accuracy:  0.25  Recall:  0.25  

In [ ]:
best_index = smoothing_parameter_results.index(max(smoothing_parameter_results))
print("Akurasi tertinggi :", smoothing_parameter_results[best_index])
print("Presisi tertinggi :", smoothing_precision[best_index])
print("Recall tertinggi :", smoothing_recall[best_index])
print("Smoothing parameter:", std_lists[best_index])

In [ ]:
print("Akurasi:")
print(smoothing_parameter_results)
print("Presisi:")
print(smoothing_precision)
print("Recall:")
print(smoothing_recall)

# UJI SKENARIO KEDUA - KOMBINASI SUDUT GLCM

Pada uji skenario kedua, akan dilakukan pengujian terhadap kombinasi sudut GLCM. Dimana dilakukan pengujian terhadap 4 sudut GLCM yaitu 0, 45, 90, dan 135. Untuk jarak digunakan kombinaasi jarak 1, 2, 3, 4, dan 5

Pada pengujian sebelumnya didapatkan smoothing parameter terbaik dengan nilai 0.1, maka untuk uji skenario sudut GLCM digunakan nilai smoothing parameter bernilai 1

In [ ]:
file_path = "GLCM extract/Combined/Non-Zoning/"

In [ ]:
glcm_results = []
glcm_recall = []
glcm_precision = []
file_list = []
for file_name in os.listdir(file_path):
    file = file_path+file_name
    print(file)
    datasets = pd.read_csv(file)
    
    X = datasets.loc[:, ['dissimilarity', 'homogeneity', 'entropy']].values
    y = datasets.loc[:, 'labels'].values
    X = preprocessing.normalize(X)
    
    acc = 0
    prec = 0
    rec = 0
    counter = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pnn = algorithms.PNN(std=0.1, verbose=True)
        pnn.train(X_train, y_train)

        y_predicted = pnn.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_predicted)
        precision = metrics.precision_score(y_test, y_predicted, average='micro')
        recall = metrics.recall_score(y_test, y_predicted, average='micro')
        counter+=1
        acc+=accuracy
        prec+=precision
        rec+=recall
        print("Fold: ", counter, ", Accuracy: ",accuracy, " Recall: ",recall, " Precision: ",precision)
        print("-----------------------------------------------------------------------------------------------------------")
    acc=acc/counter
    prec=prec/counter
    rec=rec/counter
    print("\n\n\n\n\n\n")
    print("Average accuracy: ", acc)
    glcm_results.append(acc)
    glcm_recall.append(rec)
    glcm_precision.append(prec)
    file_list.append(file_name)
    print("=========================================================================================================")

In [ ]:
file_index = glcm_results.index(max(glcm_results))

In [ ]:
print("File name for best accuracy: ", file_list[file_index])
print("Accuracy: ", glcm_results[file_index])
print("Precision: ", glcm_precision[file_index])
print("Recall: ", glcm_recall[file_index])

In [ ]:
for file_index in range(len(file_list)):
    print("FZile: ", file_list[file_index], " Acc: ", glcm_results[file_index], " Prec: ", glcm_recall[file_index], " Rec: ", glcm_precision[file_index], )

Dari uji skenario di atas, didapatkan sudut dan jarak GLCM terbaik yaitu sudut 0 dan jarak 1 dengan akurasi 0.21940603756193106

# UJI SKENARIO KETIGA - ZONING

Pengujian skenario ketiga yang dilakukan adalah zoning, dimana akan dilakukan zoning dengan ukuran 2x2 dan ukuran 4x4. Untuk smoothing parameter ditentukan dengan nilai 0.1 dan variasi GLCM dengan sudut 0 dan jarak 1

Pengujian pertama dilakukan terhadap zoning ukuran 2x2

In [ ]:
file = "GLCM extract/Combined/2x2/glcm_0_0.csv"

In [ ]:
datasets = pd.read_csv(file)

In [ ]:
exclude_columns = ['contrast','ASM', 'energy', 'correlation', 'labels', 'img_paths']
select_columns = [x for x in datasets.columns if x not in exclude_columns]

X = datasets.loc[:, select_columns].values
y = datasets.loc[:, 'labels'].values
X = preprocessing.normalize(X)

acc = 0
rec = 0
prec = 0
counter = 0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pnn = algorithms.PNN(std=0.1, verbose=True)
    pnn.train(X_train, y_train)

    y_predicted = pnn.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_predicted)
    precision = metrics.precision_score(y_test, y_predicted, average='micro')
    recall = metrics.recall_score(y_test, y_predicted, average='micro')
    counter+=1
    acc+=accuracy
    prec+=precision
    rec+=recall
    print("Fold: ", counter, ", Accuracy: ",accuracy, " Recall: ",recall, " Precision: ",precision)
    print("-----------------------------------------------------------------------------------------------------------")
acc=acc/counter
prec=prec/counter
rec=rec/counter
print("\n\n\n\n\n\n")
print("Average accuracy: ", acc)
print("Average preicision: ", prec)
print("Average recall: ", rec)
print("=========================================================================================================")

Pada penggunaan zoning berukuran 2x2 diperoleh akurasi sebesar 0.6100659638207168 dengan presisi dan recall yang sama

Pengujian kedua dilakukan terhadap zoning ukuran 4x4

In [ ]:
file = "GLCM extract/Combined/4x4/glcm_0_0.csv"

In [ ]:
datasets = pd.read_csv(file)

# VISUALIZATION

In [ ]:
exclude_columns = ['contrast','ASM', 'energy', 'correlation', 'labels', 'img_paths']
select_columns = [x for x in datasets.columns if x not in exclude_columns]
a4_dims = (11.7, 8.27)
for selected_feature in select_columns:
    plt.subplots(figsize=a4_dims)
    sns.boxplot(x=selected_feature, y="labels", data=datasets)
plt.show()

# END HERE

In [ ]:
exclude_columns = ['contrast','ASM', 'energy', 'correlation', 'labels', 'img_paths']
select_columns = [x for x in datasets.columns if x not in exclude_columns]

X = datasets.loc[:, select_columns].values
y = datasets.loc[:, 'labels'].values
X = preprocessing.normalize(X)

acc = 0
rec = 0
prec = 0
counter = 0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pnn = algorithms.PNN(std=0.1, verbose=True)
    pnn.train(X_train, y_train)

    y_predicted = pnn.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_predicted)
    precision = metrics.precision_score(y_test, y_predicted, average='micro')
    recall = metrics.recall_score(y_test, y_predicted, average='micro')
    counter+=1
    acc+=accuracy
    prec+=precision
    rec+=recall
    print("Fold: ", counter, ", Accuracy: ",accuracy, " Recall: ",recall, " Precision: ",precision)
    print("-----------------------------------------------------------------------------------------------------------")
acc=acc/counter
prec=prec/counter
rec=rec/counter
print("\n\n\n\n\n\n")
print("Average accuracy: ", acc)
print("Average preicision: ", prec)
print("Average recall: ", rec)
print("=========================================================================================================")

Pada pengujian dengan zoning berukuran 4x4 diperoleh akurasi sebesar 0.808274282751469

# UJI SKENARIO KEEMPAT - PEMBAGIAN DATA

Dari skenario yang telah dilakukan sebelumnya, didapatkan beberapa parameter terbaik untuk model PNN.
Antara lain:

-smoothing parameter bernilai 0.1

-sudut glcm 0 dengan jarak 1

-zoning dengan ukuran 4x4

Pada uji skenario terakhir akan dilakukan variasi pembagian data dengan rasio train:test 9:1 hingga 1:9. Pengujian antara lain:

1:9

1:7

1:5

1:4

1:3

1:2

1:1

2:1

3:1

4:1


5:1

7:1

9:1

In [ ]:
kfold_list = [10, 8, 6, 5, 4, 3, 2]

In [ ]:
datasets_results = []
datasets_prec = []
datasets_rec = []
for kfold_val in kfold_list:
    acc1 = 0
    acc2 = 0
    prec1 = 0
    prec2 = 0
    rec1 = 0
    rec2 = 0
    counter = 0
    kf = KFold(n_splits=kfold_val, random_state=1, shuffle=True)
    for train_index, test_index in kf.split(X):
        #first ratio, ex 9:1 then reversed in next case
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pnn = algorithms.PNN(std=0.1, verbose=True)
        pnn.train(X_train, y_train)

        y_predicted = pnn.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_predicted)
        precision = metrics.precision_score(y_test, y_predicted, average='micro')
        recall = metrics.recall_score(y_test, y_predicted, average='micro')
        print("Fold: ", counter)
        print("Ratio 1:",kfold_val-1)
        print("Accuracy: ",accuracy)
        print("Precision: ",precision)
        print("Recall: ",recall)
        acc1+=accuracy
        prec1+=precision
        rec1+=recall
        
        #second ratio, reversed from case before
        
        X_train, X_test = X[test_index], X[train_index]
        y_train, y_test = y[test_index], y[train_index]
        pnn = algorithms.PNN(std=0.1, verbose=True)
        pnn.train(X_train, y_train)

        y_predicted = pnn.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_predicted)
        precision = metrics.precision_score(y_test, y_predicted, average='micro')
        recall = metrics.recall_score(y_test, y_predicted, average='micro')
        print("Fold: ", counter)
        print("Ratio ",kfold_val-1, ":1")
        print("Accuracy: ",accuracy)
        print("Precision: ",precision)
        print("Recall: ",recall)
        acc2+=accuracy
        prec2+=precision
        rec2+=recall
        
        counter+=1
        
        
    acc1=acc1/counter
    acc2=acc2/counter
    result = [acc1, acc2]
    datasets_results = np.concatenate((datasets_results, result), axis=0) 
    
    rec1=rec1/counter
    rec2=rec2/counter
    rec_result = [rec1, rec2]
    datasets_rec = np.concatenate((datasets_rec, rec_result), axis=0)
    
    prec1=prec1/counter
    prec2=prec2/counter
    prec_result = [prec1, prec2]
    datasets_prec = np.concatenate((datasets_prec, prec_result), axis=0)
    
    print("\n\n\n\n\n\n")
    print("Average accuracy for Ratio 1:",kfold_val-1, " = ", acc1)
    print("Average accuracy for Ratio ",kfold_val-1, ":1 = ", acc2)
    print("=========================================================================================================")

In [ ]:
datasets_results = datasets_results.reshape(-1, 2)
datasets_rec = datasets_rec.reshape(-1, 2)
datasets_prec = datasets_prec.reshape(-1, 2)

In [ ]:
datasets_results

In [ ]:
datasets_rec

In [ ]:
datasets_prec

In [ ]:
best_index = np.where(datasets_results == np.amax(datasets_results))
print("Akurasi tertinggi :", datasets_results[best_index])
print("K-fold parameter:", kfold_list[best_index[0][0]])

In [ ]:
kfold_list

In [ ]:
# file = "GLCM extract/glcm_0_0_zoning size=8.csv"

In [ ]:
# datasets = pd.read_csv(file)

In [ ]:
# datasets.describe()

In [ ]:
# exclude_columns = ['contrast','ASM', 'energy', 'correlation', 'labels', 'img_paths']
# select_columns = [x for x in datasets.columns if x not in exclude_columns]

# X = datasets.loc[:, select_columns].values
# y = datasets.loc[:, 'labels'].values
# X = preprocessing.normalize(X)

# acc = 0
# rec = 0
# prec = 0
# counter = 0
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     pnn = algorithms.PNN(std=0.1, verbose=True)
#     pnn.train(X_train, y_train)

#     y_predicted = pnn.predict(X_test)
#     accuracy = metrics.accuracy_score(y_test, y_predicted)
#     precision = metrics.precision_score(y_test, y_predicted, average='micro')
#     recall = metrics.recall_score(y_test, y_predicted, average='micro')
#     counter+=1
#     acc+=accuracy
#     prec+=precision
#     rec+=recall
#     print("Fold: ", counter, ", Accuracy: ",accuracy, " Recall: ",recall, " Precision: ",precision)
#     print("-----------------------------------------------------------------------------------------------------------")
# acc=acc/counter
# prec=prec/counter
# rec=rec/counter
# print("\n\n\n\n\n\n")
# print("Average accuracy: ", acc)
# print("Average preicision: ", prec)
# print("Average recall: ", rec)
# print("=========================================================================================================")